<a href="https://colab.research.google.com/github/applejxd/colaboratory/blob/master/ml/winograd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

深層学習での convolution は信号処理の correlation であることに注意.

scipy の [correlate2d](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.correlate2d.html) を使う.

In [30]:
import numpy as np
from scipy.signal import correlate2d

def normal_convolution(input_matrix, kernel):
    return correlate2d(input_matrix, kernel, mode='valid')

# 入力画像（4x4）
input_matrix = np.array([
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12],
    [13, 14, 15, 16]
])

# 3x3のカーネル
kernel = np.array([
    [1, 0, -1],
    [1, 0, -1],
    [1, 0, -1]
])

# 通常の畳み込みを実行
output_normal = normal_convolution(input_matrix, kernel)
print("通常の畳み込み結果:")
print(output_normal)

通常の畳み込み結果:
[[-6 -6]
 [-6 -6]]


Winograd 畳み込み $F(2\times2,3\times3)$ を実行.

これは出力する行列が 2x2 で、カーネルが 3x3 という意味。
逆算して入力する行列は 4x4.

公式は[こちら](https://proc-cpuinfo.fixstars.com/2017/06/winograd-minimal-filtering/)参照.

In [31]:
def winograd_convolution(input_matrix, kernel):
    # フィルタ変換行列
    G = np.array([
        [1, 0, 0],
        [0.5, 0.5, 0.5],
        [0.5, -0.5, 0.5],
        [0, 0, 1]
    ])

    # 入力変換行列
    B = np.array([
        [1, 0, -1, 0],
        [0, 1, 1, 0],
        [0, -1, 1, 0],
        [0, 1, 0, -1]
    ])

    # 出力変換行列
    A = np.array([
        [1, 1, 1, 0],
        [0, 1, -1, -1]
    ])

    # フィルタ変換
    Gg = np.dot(G, np.dot(kernel, G.T))

    # 入力ブロックの変換
    d = input_matrix[:4, :4]
    Bd = np.dot(B, np.dot(d, B.T))

    # 要素ごとの積
    Y = Gg * Bd

    # 出力変換
    Y_transformed = np.dot(A, np.dot(Y, A.T))

    return Y_transformed[:2, :2]

# Winogradアルゴリズムによる畳み込みを実行
output_winograd = winograd_convolution(input_matrix, kernel)
print("Winogradアルゴリズムによる畳み込み結果:")
print(output_winograd)

Winogradアルゴリズムによる畳み込み結果:
[[-6. -6.]
 [-6. -6.]]
